In [3]:
import pandas as pd
import numpy as np
import requests
import json
import datetime

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)


df = pd.read_csv("exchange_rates_final.csv")


current_date = datetime.date.today()


response_exchange = requests.get("http://api.nbp.pl/api/exchangerates/tables/A/{}".format(current_date))
print("Status code for exchange table", response_exchange.status_code)
response_gold = requests.get("http://api.nbp.pl/api/cenyzlota/{}".format(current_date))
print("Status code for gold table", response_gold.status_code)

if response_exchange.status_code == 200 and response_gold.status_code == 200:
    row = {"Date" : current_date}
    gold_price = response_gold.json()[0]["cena"]
    
    for currency in range(len(response_exchange.json()[0]["rates"])):
        row_key = response_exchange.json()[0]["rates"][currency]["code"]
        row_value = response_exchange.json()[0]["rates"][currency]["mid"]
        row[str(row_key)] = row_value 
    
    
    row["gold_price"] = gold_price
    new_df = pd.DataFrame([row])
    df = df.append(new_df, ignore_index=True)

Status code for exchange table 200
Status code for gold table 200


In [2]:
import requests
import json
import datetime
import awswrangler as wr

current_date = datetime.date.today()
os_input_s3_cleansed_layer = os.environ['s3_cleansed_layer']


def lambda_handler(event, context):
    
    response_exchange = requests.get("http://api.nbp.pl/api/exchangerates/tables/A/{}".format(current_date))
    print("Status code for exchange table", response_exchange.status_code)
    response_gold = requests.get("http://api.nbp.pl/api/cenyzlota/{}".format(current_date))
    print("Status code for gold table", response_gold.status_code)
    
    if response_exchange.status_code == 200 and response_gold.status_code == 200:
        row = {"Date" : current_date}
        gold_price = response_gold.json()[0]["cena"]
        
        for currency in range(len(response_exchange.json()[0]["rates"])):
            row_key = response_exchange.json()[0]["rates"][currency]["code"]
            row_value = response_exchange.json()[0]["rates"][currency]["mid"]
            row[str(row_key)] = row_value 
        
        
        row["gold_price"] = gold_price
    
    
        wr_response = wr.s3.to_parquet(
                df=new_df,
                path=os_input_s3_cleansed_layer,
                dataset=True,
            )

ModuleNotFoundError: No module named 'awswrangler'

In [5]:
df.shape

(929, 35)